<a href="https://colab.research.google.com/github/skywalker0803r/ESun2021WinterGame/blob/main/debug/debug1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def create_df():
  df = pd.DataFrame()
  df['chid'] = [10128239 for _ in range(24)]
  df['dt'] = [*range(1,25)]
  df['shop_tag'] = [ str(np.random.randint(0,50)) for _ in range(24)]
  df['txn_amt'] = [ np.random.randint(0,9999) for _ in range(24)]
  return df
df = create_df()

In [7]:
k = 10128239
dt_sample_df = pd.DataFrame(index=np.arange(1.0, 25.0, 1).tolist())
test_df = df[df['chid'] == k]
pivot_df_T = dt_sample_df.join(test_df.pivot(index='dt', columns='shop_tag', values='txn_amt').fillna(0)).T
dt_df = pd.DataFrame(index=pivot_df_T.index)
column_list = pivot_df_T.columns.tolist()
for i in column_list:
    rank_df = pivot_df_T.loc[:, [i]].rank(method='max') + 50 - pivot_df_T.shape[0]
    rank_df[rank_df == min(rank_df.values)[0]] = 1
    dt_df.loc[:, str(i)] = rank_df.iloc[:, -1].values

In [8]:
dt_df

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0
1,1.0,50.0,50.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
17,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
22,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0
25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0
26,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
27,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
28,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
29,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0


In [9]:
shop_tag_rank = dt_df.T
shop_tag_rank.columns = ["shop_tag_" + i + "_rank" for i in shop_tag_rank.columns.tolist()]

output = shop_tag_rank
for j in np.r_[1:12]:
    temp_shop_tag_rank = shop_tag_rank.shift(periods =j)
    temp_shop_tag_rank.columns = [str(i) + "_t-" + str(j) for i in shop_tag_rank.columns.tolist()]

    output = pd.concat([output, temp_shop_tag_rank], axis=1)

top5_shop_tag = pd.DataFrame()
for j in np.r_[1:6]:
    for i in range(len(column_list)):
        sort_index = np.argsort(pivot_df_T[column_list[i]].values.tolist())
        top5_shop_tag.loc[column_list[i], 'top'+ str(j)] = pivot_df_T.index[sort_index[-j]] if pivot_df_T.loc[:, column_list[i]][sort_index[-j]] > 0 else 0    
top5_shop_tag.index = output.index
output = pd.concat([output, top5_shop_tag], axis=1)
        
output['chid'] = k
#for i in ['masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam', 'gender_code', 'age', 'primary_card']:
#    output[i] = df[df['chid'] == k].loc[:,i].values[0]
output = output.reset_index().rename(columns={'index': 'dt'})

In [10]:
output

,dt,shop_tag_1_rank,shop_tag_10_rank,shop_tag_17_rank,shop_tag_20_rank,shop_tag_22_rank,shop_tag_25_rank,shop_tag_26_rank,shop_tag_27_rank,shop_tag_28_rank,shop_tag_29_rank,shop_tag_3_rank,shop_tag_32_rank,shop_tag_33_rank,shop_tag_4_rank,shop_tag_41_rank,shop_tag_43_rank,shop_tag_44_rank,shop_tag_49_rank,shop_tag_5_rank,shop_tag_7_rank,shop_tag_1_rank_t-1,shop_tag_10_rank_t-1,shop_tag_17_rank_t-1,shop_tag_20_rank_t-1,shop_tag_22_rank_t-1,shop_tag_25_rank_t-1,shop_tag_26_rank_t-1,shop_tag_27_rank_t-1,shop_tag_28_rank_t-1,shop_tag_29_rank_t-1,shop_tag_3_rank_t-1,shop_tag_32_rank_t-1,shop_tag_33_rank_t-1,shop_tag_4_rank_t-1,shop_tag_41_rank_t-1,shop_tag_43_rank_t-1,shop_tag_44_rank_t-1,shop_tag_49_rank_t-1,shop_tag_5_rank_t-1,...,shop_tag_26_rank_t-10,shop_tag_27_rank_t-10,shop_tag_28_rank_t-10,shop_tag_29_rank_t-10,shop_tag_3_rank_t-10,shop_tag_32_rank_t-10,shop_tag_33_rank_t-10,shop_tag_4_rank_t-10,shop_tag_41_rank_t-10,shop_tag_43_rank_t-10,shop_tag_44_rank_t-10,shop_tag_49_rank_t-10,shop_tag_5_rank_t-10,shop_tag_7_rank_t-10,shop_tag_1_rank_t-11,shop_tag_10_rank_t-11,shop_tag_17_rank_t-11,shop_tag_20_rank_t-11,shop_tag_22_rank_t-11,shop_tag_25_rank_t-11,shop_tag_26_rank_t-11,shop_tag_27_rank_t-11,shop_tag_28_rank_t-11,shop_tag_29_rank_t-11,shop_tag_3_rank_t-11,shop_tag_32_rank_t-11,shop_tag_33_rank_t-11,shop_tag_4_rank_t-11,shop_tag_41_rank_t-11,shop_tag_43_rank_t-11,shop_tag_44_rank_t-11,shop_tag_49_rank_t-11,shop_tag_5_rank_t-11,shop_tag_7_rank_t-11,top1,top2,top3,top4,top5,chid
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,0.0,0.0,0.0,0.0,10128239
1,2.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,10128239
2,3.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0.0,0.0,0.0,10128239
3,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0.0,0.0,0.0,0.0,10128239
4,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43,0.0,0.0,0.0,0.0,10128239
5,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,0.0,0.0,0.0,0.0,10128239
6,7.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,0.0,0.0,0.0,0.0,10128239
7,8.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,Na

In [11]:
# 為何不是49?
len(set([ f'shop_tag_{str(i)}_rank'for i in range(51)])&set(output.columns.tolist()))

20